# Import Libraries

In [5]:
!pip install tensorflow_text

import pathlib
import os
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

from transformers import TFMobileBertForQuestionAnswering, MobileBertConfig

# Define Functions

In [6]:
def define_preprocessor():
    preprocessor = hub.load("https://www.kaggle.com/models/tensorflow/bert/frameworks/TensorFlow2/variations/en-uncased-preprocess/versions/3")

    bert_pack_inputs = hub.KerasLayer(
        preprocessor.bert_pack_inputs,
        arguments=dict(seq_length=512))

    tokenizer = hub.KerasLayer(preprocessor.tokenize)

    return tokenizer, preprocessor, bert_pack_inputs


def define_necessaries():
    pretrained_name = "vumichien/mobilebert-uncased-squad-v2"
    configurator = MobileBertConfig
    model_architecture = TFMobileBertForQuestionAnswering

    return pretrained_name, configurator, model_architecture


def define_dataset(num_examples):
    with open('/content/drive/MyDrive/Colab Notebooks/Chatbot/dataset/health-care_qa.txt', 'r', encoding='utf-8') as file:
        dataset = file.readlines()

    questions = []
    answers = []

    for line in dataset[:num_examples]:
        q, a = line.split('\t')
        questions.append(q)
        answers.append(a.replace('\n', ''))

    return questions, answers


def preprocessing(question, answer):
    tokenized_question = tokenizer([question])
    tokenized_answer = tokenizer([answer])

    packed_inputs = bert_pack_inputs([tokenized_question, tokenized_answer])

    input_ids = packed_inputs["input_word_ids"]
    attention_mask = packed_inputs["input_mask"]
    token_type_ids = packed_inputs["input_type_ids"]

    return input_ids, attention_mask, token_type_ids


def create_model(pretrained_model_name, configurator, model_arch):
    configuration = configurator(
        hidden_size=512,
        hidden_act='relu',
        hidden_dropout_prob=0.2,
        attention_probs_dropout_prob=0.2,
        max_position_embeddings=512)

    model = model_arch.from_pretrained(pretrained_model_name, config=configuration)
    model.build()
    model.summary()

    return model

# Execute Functions

In [7]:
# Define preprocessor and its tokenizer
tokenizer, preprocessor, bert_pack_inputs = define_preprocessor()

# Define the model architecture and configurator
pretrained_name, configurator, model_architecture = define_necessaries()

# Define "questions" and "answers" dataset
questions, answers = define_dataset(500)

# Example of "Preprocessing Dataset"
ex_input_ids, ex_attention_mask, ex_token_type_ids = preprocessing(questions[0], answers[0])

# Define dummy labels
start_label = tf.constant([20])
end_label = tf.constant([35])

# Create model that already been configured
model = create_model(pretrained_name, configurator, model_architecture)

All model checkpoint layers were used when initializing TFMobileBertForQuestionAnswering.

All the layers of TFMobileBertForQuestionAnswering were initialized from the model checkpoint at vumichien/mobilebert-uncased-squad-v2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMobileBertForQuestionAnswering for predictions without further training.


Model: "tf_mobile_bert_for_question_answering_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilebert (TFMobileBertMa  multiple                  24581888  
 inLayer)                                                        
                                                                 
 qa_outputs (Dense)          multiple                  1026      
                                                                 
Total params: 24582914 (93.78 MB)
Trainable params: 24582914 (93.78 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


# Fine Tuning and Training

In [8]:
optimizer = tf.keras.optimizers.RMSprop(1e-3)

EPOCHS = 5

history = []

for epoch in range(EPOCHS):
    print(f"Epoch: {epoch + 1}/{EPOCHS}", end=" ")

    losses = []

    for i in range(len(questions)):
        with tf.GradientTape() as tape:
            input_ids, attention_mask, token_type_ids = preprocessing(questions[i], answers[i])
            outputs = model(input_ids, attention_mask, token_type_ids, start_positions=start_label, end_positions=end_label, training=True)

            # Calculate loss
            loss = tf.math.reduce_mean(outputs.loss)
            losses.append(round(float(loss), 2))

            # Update gradients and apply to the optimizer
            gradients = tape.gradient(loss, model.trainable_variables)
            optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    losses = np.mean(losses)
    history.append(losses)

    print(f'Loss: {losses}')

Epoch: 1/5 Loss: 74133.11196
Epoch: 2/5 Loss: nan
Epoch: 3/5 Loss: nan
Epoch: 4/5 Loss: nan
Epoch: 5/5 Loss: nan


# Export - Convert - Download Model

## Export Model

In [9]:
export_dir = "qa_model"
tf.saved_model.save(model, export_dir)

## Convert Model into TF Lite Format

In [10]:
converter = tf.lite.TFLiteConverter.from_saved_model(export_dir)
tflite_model = converter.convert()

tflite_model_file = pathlib.Path(os.path.join(export_dir, "qamodel.tflite"))
tflite_model_file.write_bytes(tflite_model)

22439552

## Download Converted Model and It's Folder

In [11]:
!zip -r /content/qa_model.zip /content/qa_model

from google.colab import files
files.download('qa_model.zip')

  adding: content/qa_model/ (stored 0%)
  adding: content/qa_model/qamodel.tflite (deflated 47%)
  adding: content/qa_model/saved_model.pb (deflated 93%)
  adding: content/qa_model/assets/ (stored 0%)
  adding: content/qa_model/variables/ (stored 0%)
  adding: content/qa_model/variables/variables.index (deflated 88%)
  adding: content/qa_model/variables/variables.data-00000-of-00001 (deflated 88%)
  adding: content/qa_model/fingerprint.pb (stored 0%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>